In [ ]:
import json
import pandas as pd
import sqlalchemy
import datetime
import requests
import pw

In [ ]:
# function to extract weather data using API
def extract_weather(list_cities):
  API_key = "YOUR_API_KEY"
  weather_dictionary = {"city_code": [],
                        "city": [],
                        "country": [],
                        "latitude": [],
                        "longitude": [],
                        "date_time": [],
                        "temperature": [],
                        "felt_temperature": [],
                        "weather": [],
                        "detailed_weather": [],
                        "risk_of_rain": [],
                        "rain":[],
                        "snow":[],
                        "wind_speed": [],
                        "retr_data":[]}
    
        
  for j in range(len(list_cities)):
    city = list_cities[j]
    weather_city = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    weather_city_json = weather_city.json()

    for i, hour in enumerate(weather_city_json["list"]):
      weather_dictionary["city"].append(weather_city_json["city"]["name"])
      weather_dictionary["country"].append(weather_city_json["city"]["country"])
      weather_dictionary["latitude"].append(weather_city_json["city"]["coord"]["lat"])
      weather_dictionary["longitude"].append(weather_city_json["city"]["coord"]["lon"])
      
      weather_dictionary["weather"].append(weather_city_json["list"][i]["weather"][0]["main"].lower())
      weather_dictionary["detailed_weather"].append(weather_city_json["list"][i]["weather"][0]["description"])
      weather_dictionary["risk_of_rain"].append(weather_city_json["list"][i]["pop"])
      
      weather_dictionary["date_time"].append(pd.to_datetime(weather_city_json["list"][i]["dt_txt"]))
      weather_dictionary["temperature"].append(weather_city_json["list"][i]["main"]["temp"])

      weather_dictionary["felt_temperature"].append(weather_city_json["list"][i]["main"]["feels_like"])
      weather_dictionary["wind_speed"].append(weather_city_json["list"][i]["wind"]["speed"])
        
      weather_dictionary["retr_data"].append(datetime.datetime.today().strftime('%Y-%m-%d-%H:%M')) 

      if "rain" in weather_city_json["list"][i].keys():
        weather_dictionary["rain"].append(pd.to_numeric(weather_city_json["list"][i]["rain"]["3h"]))
      else:
        weather_dictionary["rain"].append(0)
      
      if "snow" in weather_city_json["list"][i].keys():
        weather_dictionary["snow"].append(pd.to_numeric(weather_city_json["list"][i]["snow"]["3h"]))
      else:
        weather_dictionary["snow"].append(0)
        
        
      if weather_city_json["city"]["name"] == "Berlin":
        weather_dictionary["city_code"].append(1)
      elif weather_city_json["city"]["name"] == "Hamburg":
        weather_dictionary["city_code"].append(2)
      elif weather_city_json["city"]["name"] == "London":
        weather_dictionary["city_code"].append(3)
      elif weather_city_json["city"]["name"] == "Manchester":
        weather_dictionary["city_code"].append(4)
      elif weather_city_json["city"]["name"] == "Barcelona":
        weather_dictionary["city_code"].append(5)    
    
  return pd.DataFrame(weather_dictionary)

In [ ]:
def get_flights(ICAO_list):
  # create empty dictionary
  flights_dictionary = {"icao":[],
                        "flight_number":[],
                        "departure_icao":[],
                        "arrival_TimeLocal":[],
                        "arrival_terminal":[],
                        "airline_name":[],
                        "retr_date":[]
                        }
  today = datetime.date.today()
  tomorrow = today + datetime.timedelta(days=1)
  dates = [["00:00", "11:59"],["12:00", "23:59"]]

  querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withPrivate":"true","withLocation":"false"}
  headers = {
      "X-RapidAPI-Key": "YOUR_API_KEY",
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
      }

  # use API to retrieve data for flights
  for i,icao in enumerate(ICAO_list):
    #icao = ICAO_list[i]
    
    for da in dates:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{da[0]}/{tomorrow}T{da[1]}"
      # get data and use json_normalize
      response = requests.request("GET", url, headers=headers, params=querystring)

      if response.status_code == 200:
        flights_json_df = pd.json_normalize(response.json()["arrivals"])

        for j in range(len(flights_json_df)):
          # fill dictionary
          flights_dictionary["icao"].append(icao)
          flights_dictionary["flight_number"].append(flights_json_df["number"][j])
          flights_dictionary["departure_icao"].append(flights_json_df["departure.airport.icao"][j])
          flights_dictionary["arrival_TimeLocal"].append(pd.to_datetime(flights_json_df["arrival.scheduledTimeLocal"][j][:16]))
          flights_dictionary["airline_name"].append(flights_json_df["airline.name"][j])
          flights_dictionary["retr_date"].append(pd.to_datetime(today))

          if "arrival.terminal" in flights_json_df.columns.values.tolist():
            flights_dictionary["arrival_terminal"].append(flights_json_df["arrival.terminal"][j])
          else:
            flights_dictionary["arrival_terminal"].append("-1")  
      else:
        print(f"Code {response.status_code}. This airport {icao} doesn't exist.")
        continue;

  return pd.DataFrame(flights_dictionary)
  

In [ ]:
# function to extract sunlight data using API
def extract_light(list_cities):
  API_key = "YOUR_API_KEY"
  light_dictionary = {"city_code": [],
                        "city": [],
                        "sunrise": [],
                        "sunset": [],
                        "retr_data":[]
                      }
    
        
  for j,city in enumerate(list_cities):
    weather_city = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    weather_city_json = weather_city.json()

    light_dictionary["city"].append(weather_city_json["city"]["name"])
    light_dictionary["sunrise"].append(datetime.datetime.fromtimestamp(weather_city_json["city"]["sunrise"]))
    light_dictionary["sunset"].append(datetime.datetime.fromtimestamp(weather_city_json["city"]["sunset"]))
    light_dictionary["retr_data"].append(datetime.datetime.today().strftime('%Y-%m-%d-%H:%M')) 
        
        
    if weather_city_json["city"]["name"] == "Berlin":
        light_dictionary["city_code"].append(1)
    elif weather_city_json["city"]["name"] == "Hamburg":
        light_dictionary["city_code"].append(2)
    elif weather_city_json["city"]["name"] == "London":
        light_dictionary["city_code"].append(3)
    elif weather_city_json["city"]["name"] == "Manchester":
        light_dictionary["city_code"].append(4)
    elif weather_city_json["city"]["name"] == "Barcelona":
        light_dictionary["city_code"].append(5)    
    
  return pd.DataFrame(light_dictionary)